## This is a Draft for codes in https://www.tensorflow.org/get_started/mnist/pros

In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz


In [4]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

In [8]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [9]:
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [10]:
x_image = tf.reshape(x, [-1,28,28,1])

In [11]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [12]:
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [13]:
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [14]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [15]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_pred = tf.nn.softmax(y_conv)

In [ ]:
sess = tf.Session()

In [ ]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(200):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
prediction=tf.argmax(y_pred,1)
prediction_val = prediction.eval(feed_dict={x: mnist.test.images[0: 10], keep_prob: 1.0}, session=sess)
print("predictions", prediction_val)

In [ ]:
probabilities=y_pred
probabilities_val = probabilities.eval(feed_dict={x: mnist.test.images[0: 10], keep_prob: 1.0}, session=sess)
print ("probabilities", probabilities_val)

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline

In [ ]:
img = mnist.test.images[0]

In [ ]:
img.shape

In [ ]:
import numpy as np

In [ ]:
for i in range(0, 10):
    img = mnist.test.images[i]
    print('correct label:', np.argmax(mnist.test.labels[i]))
    print('predict label:', prediction_val[i])
    print('predict prob:', np.max(probabilities_val[i]))
    plt.imshow(img.reshape([28, 28]), cmap=plt.cm.binary)
    plt.show()

### Save model and load model

In [5]:
model_path = './MNIST.ckpt'

In [ ]:
save_path = tf.train.Saver().save(sess, model_path)
print ("Model saved in file: ", save_path)

In [17]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    tf.train.Saver().restore(sess, model_path)
    prediction=tf.argmax(y_pred,1)
    prediction_val = prediction.eval(feed_dict={x: mnist.test.images[0: 10], keep_prob: 1.0}, session=sess)
    print("predictions", prediction_val)
    probabilities=y_pred
    probabilities_val = probabilities.eval(feed_dict={x: mnist.test.images[0: 10], keep_prob: 1.0}, session=sess)
    print ("probabilities", probabilities_val)

predictions [7 2 1 0 4 1 4 9 5 9]
probabilities [[  8.18068220e-05   4.11334040e-05   3.46228735e-05   1.34565658e-03
    1.48854524e-04   5.77240244e-05   1.62972974e-05   9.93151426e-01
    2.44629860e-04   4.87781642e-03]
 [  6.60131546e-03   1.48299674e-03   8.46448004e-01   1.74781308e-02
    3.46077359e-05   2.04211771e-02   9.37566385e-02   9.06111163e-05
    1.34771997e-02   2.09279009e-04]
 [  1.20521159e-04   9.77806389e-01   1.16444391e-03   4.00960585e-03
    2.25959509e-03   3.21169896e-03   1.23773084e-03   5.49518038e-03
    2.89868517e-03   1.79617957e-03]
 [  9.73772705e-01   1.62673350e-05   1.06890286e-04   2.39913265e-04
    1.21108633e-05   8.47359828e-04   2.36822832e-02   3.12390039e-04
    3.64545005e-04   6.45469001e-04]
 [  2.33937302e-04   7.79437141e-06   1.16600066e-04   2.35907719e-04
    9.85767663e-01   1.54923066e-04   2.66511313e-04   1.33092829e-03
    8.64102098e-04   1.10215796e-02]
 [  1.38448186e-05   9.86447215e-01   4.38035408e-04   3.34791397e-